In [1]:
import sys
sys.path.append('../')
import doctable
import pprint

In [2]:
core = doctable.ConnectCore.open(
    target=':memory:', 
    dialect='sqlite'
)
core

ConnectCore(target=':memory:', dialect='sqlite', engine=Engine(sqlite:///:memory:), metadata=MetaData())

In [3]:
@doctable.table_schema
class MyContainer0:
    id: int
    name: str
    age: int

with core.begin_ddl() as emitter:
    tab0 = emitter.create_table(container_type=MyContainer0)
pprint.pprint(core.inspect_columns('MyContainer0'))

[{'default': None,
  'name': 'age',
  'nullable': True,
  'primary_key': 0,
  'type': INTEGER()},
 {'default': None,
  'name': 'id',
  'nullable': True,
  'primary_key': 0,
  'type': INTEGER()},
 {'default': None,
  'name': 'name',
  'nullable': True,
  'primary_key': 0,
  'type': VARCHAR()}]


In [4]:
with core.query() as q:
    q.insert_multi(tab0, [
        {'name': 'Devin J. Cornell', 'age': 50},
        {'name': 'Dorothy Andrews', 'age': 49},
    ])
    print(q.select(tab0.all_cols()).all())

[(50, None, 'Devin J. Cornell'), (49, None, 'Dorothy Andrews')]


In [5]:
with tab0.query() as q:
    q.insert_single(MyContainer0(id=0, name='John Doe', age=30))
    print(q.select())

[MyContainer0(id=None, name='Devin J. Cornell', age=50), MyContainer0(id=None, name='Dorothy Andrews', age=49), MyContainer0(id=0, name='John Doe', age=30)]


In [6]:
import datetime
@doctable.table_schema(table_name='mytable1')
class MyContainer1:
    name: str
    age: int
    id: int = doctable.Column(
        column_args=doctable.ColumnArgs(order=0, primary_key=True, autoincrement=True),
    )
    updated: datetime.datetime = doctable.Column(
        column_args=doctable.ColumnArgs(default=datetime.datetime.utcnow),
    )
    added: datetime.datetime = doctable.Column(
        column_args=doctable.ColumnArgs(
            default=datetime.datetime.utcnow, 
            onupdate=datetime.datetime.utcnow
        )
    )

with core.begin_ddl() as emitter:
    tab1 = emitter.create_table_if_not_exists(container_type=MyContainer1)
print(core.inspect_columns('mytable1'))
o = MyContainer1(name='John Doe', age=30)
print(o)
with tab1.query() as q:
    q.insert_single(o)
    print(q.select())

[{'name': 'id', 'type': INTEGER(), 'nullable': True, 'default': None, 'primary_key': 1}, {'name': 'added', 'type': DATETIME(), 'nullable': True, 'default': None, 'primary_key': 0}, {'name': 'age', 'type': INTEGER(), 'nullable': True, 'default': None, 'primary_key': 0}, {'name': 'name', 'type': VARCHAR(), 'nullable': True, 'default': None, 'primary_key': 0}, {'name': 'updated', 'type': DATETIME(), 'nullable': True, 'default': None, 'primary_key': 0}]
MyContainer1(name='John Doe', age=30, id=MISSING, updated=MISSING, added=MISSING)
[MyContainer1(name='John Doe', age=30, id=1, updated=datetime.datetime(2023, 11, 14, 15, 3, 48, 302621), added=datetime.datetime(2023, 11, 14, 15, 3, 48, 302618))]
